In [2]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
from scipy.stats import linregress

# Folder .nc
path = r"C:\Irsyad Fadillah\Bahan\Try\Agis\Data\PERMUKAAN LAUT RELATIF\data_sla"
file_list = sorted(glob.glob(os.path.join(path, "dt_global_twosat_phy_l4_*.nc")))

# Gabung semua file
ds = xr.open_mfdataset(file_list, combine='by_coords')

# Ganti nama variabel SLA sesuai data kamu
sla = ds['sla']  # Ganti kalau variabel lain

# Koordinat per desa
desa_list = {
    "Anyar": (-6.05089460, 105.91452885),
    "Cikoneng": (-6.06758559, 105.88502323),
    "Tambang_Ayam": (-6.09088559, 105.87978756),
    "Bandulu": (-6.10593517, 105.87870931)
}

# Output folder
output_folder = "output_sla_per_desa"
os.makedirs(output_folder, exist_ok=True)

for desa, (lat, lon) in desa_list.items():
    # Interpolasi titik koordinat desa
    sla_site = sla.interp(latitude=lat, longitude=lon)

    # Konversi ke DataFrame
    sla_df = sla_site.to_dataframe().dropna().reset_index()
    sla_df['year_float'] = sla_df['time'].dt.year + sla_df['time'].dt.dayofyear / 365

    # Hitung tren linear jangka panjang
    slope, intercept, *_ = linregress(sla_df['year_float'], sla_df['sla'])
    sla_df['trend'] = intercept + slope * sla_df['year_float']
    slope_mm = slope * 1000  # konversi ke mm/tahun

    # Simpan grafik tren jangka panjang
    plt.figure(figsize=(10, 5))
    plt.plot(sla_df['year_float'], sla_df['sla'], label='SLA Observasi', color='blue', alpha=0.6)
    plt.plot(sla_df['year_float'], sla_df['trend'], label=f'Tren Linear ({slope_mm:.2f} mm/tahun)', color='red')
    plt.title(f"Tren SLA di Desa {desa} (1993–2023)")
    plt.xlabel("Tahun")
    plt.ylabel("SLA (m)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    tren_grafik_path = os.path.join(output_folder, f"tren_sla_{desa}.png")
    plt.savefig(tren_grafik_path)
    plt.close()

    # Resample ke rata-rata tahunan
    sla_df['time'] = pd.to_datetime(sla_df['time'])
    sla_df.set_index('time', inplace=True)
    sla_df_yearly = sla_df['sla'].resample('Y').mean().to_frame()
    sla_df_yearly.index = sla_df_yearly.index.year
    sla_df_yearly['sla_change_mm'] = sla_df_yearly['sla'].diff() * 1000

    # Simpan grafik perubahan tahunan
    plt.figure(figsize=(10, 5))
    plt.bar(sla_df_yearly.index[1:], sla_df_yearly['sla_change_mm'][1:], color='skyblue')
    plt.axhline(0, color='gray', linestyle='--')
    plt.title(f"Kenaikan SLA Tahunan di {desa} (mm)")
    plt.xlabel("Tahun")
    plt.ylabel("Perubahan SLA (mm)")
    
    # Label per bar
    for x, y in zip(sla_df_yearly.index[1:], sla_df_yearly['sla_change_mm'][1:]):
        plt.text(x, y + 0.2, f"{y:.1f}", ha='center', va='bottom', fontsize=8)

    plt.grid(True, axis='y')
    plt.tight_layout()
    tahunan_grafik_path = os.path.join(output_folder, f"perubahan_sla_tahunan_{desa}.png")
    plt.savefig(tahunan_grafik_path)
    plt.close()

    # Gabungkan data tahunan & tren slope ke Excel
    output_excel_path = os.path.join(output_folder, f"sla_desa_{desa}.xlsx")
    with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
        sla_df.to_excel(writer, index=False, sheet_name='SLA Harian')
        sla_df_yearly.to_excel(writer, sheet_name='SLA Tahunan')
        pd.DataFrame({'slope_mm_per_year': [slope_mm]}).to_excel(writer, sheet_name='Tren Linear')

    print(f"{desa} selesai ➜ Slope: {slope_mm:.2f} mm/tahun | File: {output_excel_path}")


Anyar selesai ➜ Slope: 4.22 mm/tahun | File: output_sla_per_desa\sla_desa_Anyar.xlsx
Cikoneng selesai ➜ Slope: 4.25 mm/tahun | File: output_sla_per_desa\sla_desa_Cikoneng.xlsx
Tambang_Ayam selesai ➜ Slope: 4.25 mm/tahun | File: output_sla_per_desa\sla_desa_Tambang_Ayam.xlsx
Bandulu selesai ➜ Slope: 4.25 mm/tahun | File: output_sla_per_desa\sla_desa_Bandulu.xlsx
